Я выбрала для оценки отзывы на турецкие отели на сайте Tripadvisor.

In [1]:
import nltk # скачиваем всё необходимое
nltk.download("punkt")
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=a2f0fad23ecb6ddc6b3e3ef677bbf292882ca771fe6731f98046e5b084f9ac53
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


Я пыталась собрать отзывы путём парсинга: сначала собрать ссылки на странички отелей со страницы поиска, а затем проходиться по ним, собирая отдельно отрицательные и положительные отзывы, но поскольку на сайте на каждой странице отображается только 5 отзывов, а также нет простого выбора положительных или отрицательных отзывов, это всё заняло очень много времени и я решила оставить вариант с ручным вводом отзывов (поэтому ниже много-много текста, простите...)

In [2]:
positive_reviews = ['Отдых в отеле,несмотря на некоторые минусы, мне очень понравился, напишу, что важно именно для меня. Также хочу сделать акцент на том,что это городской отель. 1. Клиентоориентированность, заселились раньше времени, ждать не пришлось, персонал отличный, все любезны, помогают, официанты прекрасные, обслуживание и уборка выше всяких похвал. 2. Размещение- хороший чистый номер с очень удобной кроватью, матрасы и подушки великолепные, я спала как младенец. Понравился балкон, просторный и удобный. 3. Питание и алкоголь были хорошими, всегда можно выбрать блюдо на свой вкус, только соусов маловато. 4. Пляж галечный - (для городского отеля) прекрасен , вода чистая, лежаки, зонты,питание и напитки весь день, хорошего качества, полотенца можно взять прямо на пляже и там же оставить. На лифте доехать 5 минут, иногда бывает скопление людей, приходится ждать. 5. Зона СПА чистая,приятно провести время в хаммаме. Бассейн достаточно большой. 6. Бесплатные велосипеды, прокатиться по набережной, это особое удовольствие. 7. Расположение самое лучшее, близко к старому городу, набережной и торговому центру. Минусы: 1. Отель не новый, номера морально устарели и немного уставшие, но в целом это не критично. 2. Брали номер с видом на море, вид оказался боковым, а не прямым. Всё равно красивый, поэтому не стала менять. 3. Лифт на пляж приходилось иногда ждать. Общее впечатление от отдыха в любом случае хорошее, хочется вернуться.', 'Прекрасный отель, супер чистый, нам дали номер на 7 этаже с двумя прекрасными видами на море и на парк. Питание и размещение было превосходно, приятная теплая вода в бассейне, а также удивительная возможность посетить парк Легенд в Белеке. От отеля выезжают автобусы до Парка Легенд и обратно. Отдельная огромная благодарность Анастасии, которая помогала нам во всех вопросах и предлагала лучшие варианты и была всегда на связи. Мы были с семьей и дети и мы были в восторге!', 'Отличный пляж, красивый вид. Спасибо официантке в главном ресторане Ceyla, очень хороший сервис и обслуживание. Только она за нами хорошо ухаживала. Ставим за ее работу пять звезд! Удобное расположение, близко до всех торговых центров и туристических мест.', 'Останавливаемся в этом отеле уже четвертый раз, с января 2023 года. Хочется отметить замечательный персонал отеля, особенно Марию Лаптеву, она помогала нам по любым вопросам, всегда доброжелательна, внимательна, компетентна. Вообще весь персонал заслуживает высокой оценки. Хочется отметить удобную локацию отеля, рядом трамваи, свой пляж, набережная рядом, есть, где погулять. Вернемся сюда обязательно!', 'Мы отдыхали в этом отеле в сентябре 2022 года, в этом году приехали второй раз, очень понравилось. Из плюсов: большая зелёная территория, длинный песочный пляж, вкусное разнообразное питание, удобное приложение, где можно сделать заявку или посмотреть афишу мероприятий в разных зонах отеля. Отличная инфраструктура для колясок и детей. Замечательная развлекательная программа для взрослых и детей.', 'Были в ресторане 10 сентября 23г нам всё очень понравилось. Подача, обслуживание, отношение - всё на высшем уровне. Очень правильное и вкусное приготовление блюд, получили истинное удовольствие. Шеф-повар 5* Спасибо за приятный вечер Серкану и Аяне.', 'Хороший отель для семейного отдыха. Доброжелательный и внимательный персонал. Отличный номер с двумя спальнями для семьи с детьми. Достаточно разнообразное питание (голодными остаться трудно). Есть поздние завтрак и ужин. По общению с коллегами по отдыху-многие перебрались в этом сезоне из района Кемера. Пляж песчаный. Из-за этого вода мутная. Море мелкое в этом месте. - Отель находится в изоляции от курортной зоны, поэтому выехать можно на такси или с экскурсиями от официальных туроператоров. Пытались заказать экскурсии через интернет, пришли отказы из-за удаленности данного отеля от "цивилизации".', 'В отеле много ресторанов, еда вкусная и разнообразная, поесть можно практически в любое время, несколько бассейнов, есть как детские, так и взрослые бассейны. В этом отеле взял впервые урок гольфа, тренер Gökhan отлично объяснял, как нужно правильно держать клюшку, как стоять, объяснял все ошибки и я смог с первого занятия почувствовать правильную позицию для удара и игры, также показал достаточно неплохие результаты для первого раза. В общем рекомендую учится гольфу тут.', 'Замечательный отель, новый, есть все для хорошего времяпрепровождения. Шикарные рестораны на территории отеля, богатая программа. Отдельное спасибо людям, которые помогли мне провести замечательно время Тугба, Саида, Фуля и Ядигяр.', 'Менеджмент отеля на высоте! Встреча, оформление - моментальные! Номера стильные, комфортные. Обслуживание, уборка - 5+! Питание - восторг! Море, пляж - сказочно! Спасибо всему персоналу, обязательно всем будем рекомендовать! Приедем еще непременно! До встречи!', 'Прекрасный отдых. Начиная с заселения сразу (хотя прилетели посреди ночи) Уборка номеров идеальная! Питание, развлечение все на 5. Ребенок был в восторге от детского клуба огромное спасибо Дженнет, Марии и Арда. Не хотелось уезжать, теперь скучаем🥲', 'Огромное спасибо всей команде отеля Lara Barut Collection! Благодаря Вам наш отдых превзошел все наши ожидания! Сервис на высоте! Очень вкусные блюда без глютена и богатый выбор ( для нашей семьи это очень актуально ), муж был в восторге от возможности заказать безглютеновое пиво. Трепетное отношение и внимание к каждому нашему пожеланию. Великолепная анимация! Шоу, живая музыка - восторг! Везде очень чисто и красиво. Номер в лагуне был чудесен! Все продумано до мелочей ( в номере есть утюг и гладильная доска, утюжок для выпрямления волос и многое другое). Мы в восторге! А детям особенно понравился детский клуб! Там работают замечательные аниматоры, которые любят деток и свою работу! Спасибо Вам огромное! Особенно хочется поблагодарить Марию, Susu, Cennet, Dilek, Sevī, Tūlī, Arda, Yagmur, Nazli ( надеюсь мы правильно написали все имена 🙏🏻 ) Амелия и Алисия уже скучают по Вам и мечтают вернуться снова! ❤️', 'Очень хороший отель с доброжелательным персоналом, отличный spa центр Egna, хорошие массажисты, рекомендую массажиста zulikof. Удобное месторасположение. Относительно недалеко два крупных торговых центра. Питание в отеле приемлемое', 'Шикарный отель, доброжелательный персонал, отдельное спасибо Лауре, которая работает за стойкой регистрации за помощь. Очень удобный пирс, тенты, и сразу сход в море в чистую воду. вкусная пища в изобилии, чистые номера и недалеко от аэропорта.', 'Отдыхали в отеле с семьей с 1 по 8 сентября 2023 года. Ставлю отелю оценку 5 из 5! Качество обслуживания на высоте, персонал приветливый, дружелюбный. Немаловажным критерием оценки отеля является благоустройство территории: огромная территория с большим количеством развлечений, комфортные номера, разнообразное питание, ежедневная уборка номеров и пополнение мини бара, вечерние шоу, живая музыка. Отдельное спасибо персоналу и команде анимации отеля, в особенности аниматорам Кристине и Анне. Данный отель рекомендую, обязательно приеду ещё раз. Спасибо отелю Swandor за возможность отлично провести время.', 'Отличный отель,чистый пляж,хорошая кухня,приветливый персонал.Уютные номера. Каждый день проводятся ивенты от футбола до бирпонга.Много русскоговорящих аниматоров и барменов.Анимация и ивенты на высшем уровне.', 'Отдыхом остались довольны. Ребенок счастлив больше всех Ему было чем заняться В мини клубе хорошая дружелюбная атмосфера, все аниматоры хорошие , смотрят за детьми . Одевают специальные браслеты , чтобы никто не вышел из мини клуба. На территории мини клуба есть карусели , которые работают в определенное время Каждый вечер мини диско и шоу детское, потом шоу для взрослых Вернёмся ещё Отель классный , кормят вкусно , территория большая. 5 звезд', 'Были в этом отеле в августе. Очень остались довольны. Детей в мини - клубе можно оставлять с 4 лет без родителей. На территории есть мини - клуб для детей до 4 лет , есть мини - клуб для детей с 4 лет и клуб для подростков. Каждый час разные Активити в мини - клубе для деток. Постоянно какие -то воркшопы, занятия вроде лепки из глины , бабл фестиваль. Богем пати устраивают для взрослых , игры на бассейне 1 раз в неделю или 2. Шоу классные. Аниматоры все дружелюбные. С детьми постоянно занимаются. Мой ребенок остался доволен', 'Нам очень понравился ic green. Были два года подряд сентябрь 2021 и сентябрь 2022 году . В этом году опять собираемся в этот отель , тк не нашла лучше цена/ качество . Отлично встретили , заселили в номер с шикарным видом.Вкусная еда. Для деток много есть интересных локаций.', 'Отдыхала в отеле с 28 августа по 3 сентября. Очень понравилась территория отеля, большая и ухоженная. Персонал прекрасный, вежливый и всегда готовый помочь. Большое спасибо Наталье, которая помогала нам с такси из отеля в город. Девушки аниматоры, которые проводили аква аэробику- большие умнички и красавицы. Очень хорошее впечатление об отеле, буду рекомендовать его друзьям.', 'Номера современные, мебель не уставшая, есть работающий кондер, телевизор, мини-бар, бесплатный интернет. При заказе номера сразу оговорила, что мне нужен красивый вид, в итоге каждое утро встречали с мужем на балконе с видом на горы и море, тут же провожали закат с бокалами вина в руках. Из минусов - не слишком качественная уборка, но мне кажеться, что это проблема многих турецких отелей (или я слишком требовательна))). Но в принципе персонал отзывчивый и внимательный - например, вот такая ситуация была: мы приехали на два часа раньше времени заселения, но нас заселили без проблем, это очень понравилось, так как мы были уставшие. Что по цене - конечно, цены немного кусаются, подскочили по сравнению с прошлым годом, но немного утешает кэшбэк 10%, который получила от Тинькофф банка при бронировании отеля через их сервис, также получала повышенный кэшбэк при ужинах в ресторанах, за месяц набежало около тысячи, неплохо.', 'Понравилось что в отеле чисто, еда вкусная, приветливый персонал. Рыбный ресторан Аля карт вкусный. Хорошая анимация, аниматор Nurgul очень приятная и веселая. Из минусов душ в отеле, не удобный в использовании. При заезде не работал сейф. В общем отель хороший. Цена соответствует качеству. Кому нужно остановиться в центре Анталии - рекомендую.', 'Мне очень понравился отдых В отеле . Особенно работа персонала . Аниматоры не давали скучать , очень и очень приятное общение. Мы стали друзьями . Если появлялись проблемы на ресепшене все решалось в 1 момент . Бассейн чистый , море рядом , тренажерный хороший . Еда вкусная и ее много . Бармен запомнил , что я пью . Особенно хочу поблагодарить аниматора Mert ,Busra, Nurgul , Nimet.', 'Все очень понравилось, чистота еда, аниматоры, отличное расположение ,отдельно хотел бы выделить Mert аниматор, просто лучший в этом отеле , таких людей еще не встречал, огромное ему спасибо….Отель имеет свой пляж, там есть бар, бармены супер, вообще я доволен, обязательно вернусь еще !', 'Хорошее соотношение цена/качество. Персонал готовый всегда помочь) Аниматтры норм. Хорошо подойднт семейным) еда норм. Жалко не дают кальян. Хорошее расположение) море чистое. Рядом с центром. Вокруг хорошая инфраструктура.', 'Замечательное место для отдыха, приезжаем уже второй год подряд. Очень нравится расположение отеля а так же выход к морю удобный. Есть достаточно много развлечений и в самом отеле но если пойти гулять то можно также найти удачные места для ужина и просто интересные места. Замечательные уборщицы особенно Fadime всегда улыбчивая и приятная женщина, очень поднимает настроение.', 'Приехали в 10 вечера в отель. Заселили моментально. Вид из окна - лотерея. В прошлом году брали номер в старом корпусе с видом на море, моря было не видно. В этом году выбрали новый корпус с видом на город. Получили вид на главный корпус и море В первую ночь спали с включеным телевизором, пульта не было, также вешалок и теплых одеял. На след день все вопросы быстро решились Уборка Общий вид номера чистый, но полы не промывают Полотенца Некоторые пора заменить. Пляжные полотенца дают раз в день. В спортзале полотенца пляжные, что неудобно и часто их просто нет Минибар В прошлом году в номере была продукция за дополнительную плату. В середине отдыха мы обнаружили распечатанный товар. Проблему решили, но не сразу. Английский в помощь, особенно, если ночной выезд. В этом году товара extra не было. Минибар пополнялся ежедневно. Но сотрудники НЕ В ФОРМЕННОЙ ОДЕЖДЕ!! Питание Без изысков, но все вкусно Персонал Очень приветливый. Все стремятся понять, помочь даже без знания английского. Не видели конфликтов, грубости. Ежедневно работают 3 спасателя. Очень спортивные и позитивные ребята. На рецепции и спа есть русскоговорящие девушки. Всегда готовы помочь. Особенно хотелось отметить: работников lobby бара, Ferhan, она встречает вас в ресторане на входе, Goksel, работает на пляже и Furkan, официант, который обслуживал нас в течение отдыха. Очень внимательный, позитивный, с хорошим знанием английского. Вам повезет, если он будет вас обслуживать. Огромное спасибо всему персоналу отеля!', 'Пишу сразу, что данный отзыв отражает мое личное мнение. Итак, июль 2023 г. Отдых в Рамаде Плаза в 3 (третий) раз, 10 ночей, жили в делюкс, частичный вид на море и на город. С большим удовольствием отдохнули в очередной раз в этом отеле. Первый раз были в ковидном 2020, потом в 2022 и в 2023. Этот отель выбираем за шикарное море, вкусную еду, хорошие номера, очень доброжелательный персонал, отличное месторасположение отеля. Ну и конечно, за великолепные виды на море, бухту и горы. И почти нет детей, мои дети выросли и теперь хочется тишины и релакса. И отмечу, что в отеле не было пьяных, очень малопьющий отель))), нам такое подходит. Заселение. Всё четко, очень быстро. Прилетели рано, пошли на завтрак, переоделись и на пляж. Номер попросила повыше, дали 10 этаж, левая сторона, если смотреть на море. Самая лучшая сторона, солнце на балконе только до обеда. Номер. Делюкс большой номер, для двоих отлично, очень просторно. Чистый, ухоженный, все работало, отличные матрасы, постельное белье. В 2022 году отдыхала одна, номер был стандарт, жила в маленьком корпусе, 5 этаж, виды из номера не очень))), но мне было комфортно, но для двоих может быть тесновато. Уборка. Вообще в этот раз не было нареканий, повезло с уборщицей (если администрация отеля прочитает отзыв, уборщице за номер 2002 благодарность!). Еда. Всё вкусное, свежее, мясо, рыба, и прочее в ассортименте! Десерты отдельная тема, остановиться сложно, очень вкусно. Фрукты – арбузы, дыни, черешня, сливы, абрикосы, персики, нектарины, яблоки, груши и цитрусовые. Видели на завтрак гречку, но она плавала в воде). Обслуживание. Доброжелательный и милый персонал. Часть официантов помню с прошлого года, персонал тоже меня узнал, было приятно. Чаевые даю, они это заслужили, отличная работа! Пляж и море. Великолепнейшее, чистейшее море. Удобные террасы с лежаками, наша любимая – средняя, над морем она, и не так жарко, снизу поддувает. Бар, ресторан для перекусов – вкусно, удобно. Проблем с лежаками не было никогда. В бассейне не купаемся. Возле бассейна открыли еще один перекусочный ресторан, там же гезлеме и айран. Рекомендую для спокойного отдыха, релакса, прогулок и шопинга. Все рядом, в пешей доступности. До тц Марк Анталия пешком, на автобусе 08 ездили в тц Терра Сити. Итог. Поеду ли еще раз? Да!', 'прекрасный отель недалеко от старого города и с отличным пляжем. обслуживание номеров хорошее. Персонал гостиницы идет навстречу пожеланиям гостей. Особо хотелось бы отметить русскоговорящих девушек на ресепшн. Сусанна вежливая, внимательная, готова помочь в любой момент, очень помогла в налаживании общения. Отдельное спасибо ей. Проживал в отеле 20 дней с 12 июня по 01 июля. По моей просьбе на третий день поменяли номер, чем я остался доволен. Отдых был отличным. Рекомендую отель для отдыха, а также для проведения мероприятий, конференций, так как совмещал работу и отдых. Спасибо за прекрасно проведенное время!', 'Отель превзошел наши ожидания. Во-первых, идеальное расположение - в пешей доступности от старого города, магазинов, ресторанов. И в то же время на берегу - отмечу, что вода здесь намного чище, чем на городских пляжах Коньялты или Лары. Да, это спуск по лесенки, но зато очень живописно и классные понтоны, на которых можно загорать. Отдельно хочется отметить работу СПА (все было супер) и команду ресторана. Каждый вечер был какой-то сюрприз, тематическая вечеринка, дни национальных кухонь, приветственные напитки и другие интересные сюрпризы для гостей. Старались удивить! Сама еда отличая, всегда большой выбор, в том числе для тех, кто придерживается здорового питания.']

In [3]:
negative_reviews = ['Самый отвратительный отель, в котором я была. Начну с того, что это была моя тридцатая поездка в Турцию, отдыхали в различных отелях от Туи до Регнума, но такого гадюшника не видела никогда. Отель БЕЗУМНО старый… такое ощущение, что заехала в московскую тройку, вот честно. В ресторане выбор еды неплохой, но когда принялись к еде, то поняли, что выпечка несвежая. Персонал отвратительно относится к гостям. Такое ощущение, что это они приехали на отдых, а не мы. Села за столик у бара, никто не подошел за 10 минут, подошла на бар, бармен яро делал вид, что никого за баром не стоит (а я одна была), после чего сказал «мне надо отойти» и убежал в неизвестном направлении! После жалоб менеджерам, они предложили переселиться нам в лучшие номера с видом на море, но это бы не спасло этот ужасный отель) В этот же день было принято решение скорейшего переселения из отеля, уехали отдыхать в любимый Вояж и отпуск был спасен)) Если хотите просто выбросить деньги на ветер за бренд- вам сюда', 'Добрый день! Хочу поделиться своим отзывом проживания в Rixos Downtown Antalya. Номера маленькие и старые. Телевизор маленький и возраста наверно больше 10 лет. Вода в душе с плохим напором. Чтобы зайти в душ вам нужно закрыть входную дверь в ванную. Также как в маленьком отеле в историческом центре какого-нибудь города Европы. Еда на ужине не разнообразная для уровня такого отеля. Один раз были морепродукты - очень было невкусно. В этом отеле получается делать вкусное мясо и полный ужас с морепродуктами. На завтраках нужно бегать за сотрудниками, чтобы принесли кофе. Персонала явно не хватает, поэтому можете получить кофе уже после завтрака. Кстати о кофе- он невкусный из автомата с порошковым молоком. Конечно же позор для такого отеля. Интерьер отеля старый и сильно замученный временем. Ощущение, что попал в отель 1980 года. Персонал и руководство отеля не улыбаются и не приветствуют. Два года назад был на конференции в отеле Ramada Plaza by Wyndham. Отели очень похожи, вот только от Rixos я ожидал много больше. Зачем переплачивать, если в Анталье много аналогичных отелей и сильно дешевле. Не смотрите на название Rixos отеля - это отель совсем не 5 звёзд с низким уровнем сервиса.', 'Только вернулись из данного отеля. Ну что сказать, отель-просто жесть! Никак не ожидали такого от Rixos!!! Но все по порядку. Заселение. Прилетели ранним утренним рейсом. Гид сказал, что отели полупустые, номер должны дать сразу. Но странный молодой человек на ресепшене, сказал, что все забронировано, отель полный, свободных номеров нет и через сколько освободятся он не знает. Я спросила,»во сколько тогда нам приходить? В 2? Он ответил,что да,не раньше двух. Ну ок, все уставшие побрели пить кофе. Через час на ресешнене появилась русскоговорящая девушка Мария,и номера каким-то чудесным образом нашлись для всей нашей группы из 10 прибывших! За что ей огромное спасибо!!! Время, кстати, было около 10 часов!! В общем, номера свободный были,но нам почему-то сказали, что нет. Ищите Марию, она сразу же вам найдет номер и решит все проблемы! Номер. Ну что сказать,номер, конечно, старый. Ремонт хоть и был в 2011 ,но по ощущениям отель не ремонтировали с момента постройки(1990г). Кровать не была застелена покрывалом и подушек декоративных тоже не было(все лежало в шкафу). Мелочь, но хотелось бы видеть номер, оформленный как на фото. Выглядел бы намного приятнее. Изголовье кровати затертое от голов предыдущих жильцов. Прислоняешься головой, и представляешь все тех, кто делал это до тебя. Очень неприятно. Ковролин, как и писали, весь в пятнах. Похоже, химчистку не делали со времен постройки. Запах в номере- как у бабушки на антрисолях. Но самое главное-это шум с улицы!!! Нам достался угловой номер с видим на море, улица была слева от нас. Но шум ни днем,не ночью не умолкал! Если хотите спать с открытым балконом-то это вряд ли получится! Даже с закрытым очень шумно! Я даже представить боюсь, что происходит в номерах с прямым видом на улицу!!! Если хотите тишины и покоя (ну и плюс конечно хороший вид)-доплатите за номер с видом на море!!! Тут уже вопрос не только вида из номера, а больше тишины и спокойного сна!!! Убирается в номерах всего одна горничная…поэтому периодически забывает пополнять бар, может оставить 6 гелей для душа вместо шампуня и бальзама…Туалетная бумага осталась от предыдущих жильцов, ее даже не поменяли. Наклейки «Продезинфицировано» на унитазе не было ( во всех отелях это делают!! Возможно, ее просто забыли или ее нет вообще в отеле, но впечатление, что унитаз не продезинфицирован).Претензий к ней нет,но вопрос к отелю:» Почему всего лишь одна горничная на весь этаж??». Это очень тяжелая работа и один человек не может ее выполнять..Поэтому качество страдает. Халаты и тапочки были. Халаты очень тонкие, вафельные.Не пушистые махровые как в хороших отелях. Питание/ресторан. В ресторане-проходной двор. Никто тебя не приветствует(опять же как это бывает в дорогих отелях). Пару раз стояла девушка на входе и с унылым видом избирательно предлагала шампанское. Ресторан сделан очень неудобно-везде толпится народ с тарелками, столик найти сложно, бегают официанты вокруг, орут дети, мамашки с колясками, даже на ужин люди приходят в шортах и сланцах. Насладиться ужином в приятной спокойной атмосфере не получится Девочки, не везите с собой вечерние наряды! Это место не для этого!!! Тарелки тоже вовремя не убирают. Надо звать официанта, иначе можно просить до конца ужина в окружении тарелок и никто не подойдет. В первый раз столкнулась с тем, что в отеле 5 звезд на завтрак невозможно сделать тост!!!! Специальной печки для тостов или хотя бы тостера там просто НЕТ!!!! Поэтому насладится хрустящим тостом за завтраком не получится! Администрация отеля, может быть вы сможете «разориться» и предоставить гостям вашего отеля возможность делать тосты в дальнейшем?? За такие деньги хотя бы тостер могли поставить! По меню: красная рыба за все 10 дней была только раз…. Всего много, но приготовлено как-то без души. Овощи и мясо нарезаны огромными кусками. Никто не постарался сделать аккуратные дольки. Все какое-то жирное, масляное и безвкусное. Выпечка разных видов, но вся из одного теста . Только разных цветов))) Не ела вообще. С фруктами конечно был облом. Такого «ассортимента» от Турции я не ожидала: яблоки, груши, апельсины, грейпфруты, киви. Все…все жесткое и кислое. Но самое главное впечатление от ресторана-это КОТЫ!! Грязные, плешивые, наглые, орущие, вонючие! Они везде!!! Даже на территории ресторана!!! Н кто их не прогоняет! Сидишь в окружении котов, которые попрошайничают и даже не реагируют, когда их прогоняешь!! А самые «одаренные» посетители бросают их обьедки со своего стола прям на пол!🤦‍♀️ Я уже молчу про то, что насладится едой на летней веранде просто не получится! Стоит вам отвернутся, как в вашей тарелке окажется кот!!! В ресторане они (слава богу) на столы не залезают, но вот на летней веранде это происходит постоянно!! Рядом находится клумба, на которую все эти животные ходят в туалет!!!! Администрация отеля вообще не предпринимает никаких действий! Доя них это нормально!!! Сколько раз видела котов на столе!!! Непонятно, как хорошо потом после них моют тарелки! Ведь из этих же тарелок мы с вами потом будем есть!!! Я считаю это нарушением санитарных норм!!!!! Никакой отель, а особенно отель класса Rixos такого допускать не должен! Причем коты везде!!! В лобби все провоняло кошачьей мочой!! Лобби: старая убитая мебель с затертой обивкой. Тот же самый грязный ковролин, пропитанный запахом кошек ...Читала, что люди называют этот отель пафосным и шикарным…Чуть не подавилась…Видимо, раньше кроме отлей 2-3 звезд ничего лучше не видели. Кстати, отели 3х звезд могут быть намного лучше!!! Чище, новее, свежее, с современным дизайном и отличным сервисом! Но это все не про Rixos!! Возможно, он был таким 30 лет назад, но руководство ничего не делает для того, чтобы поддерживать статус данного отеля!!! Хотя бы элементарный косметический ремонт сделайте!!!! За такие деньги, которые мы платим!!! Но я понимаю, что делать ничего нп будут, потому что люди все равно едут, т.к. это Rixos и т.к. расположение у отеля одно из самых выигрышных! 10 минут до старого города на машине, 5 минут до пляжа. Есть лифт, который работал ровно 3 рада за все 10 дней. Поэтому на него особо не рассчитывайте если у вас проблемы со здоровьем. Придется спускаться и подниматься по лестнице! СПА: спустившись на минус первый этаж и пройдя по длинному темному коридору ( где вы сможете насладиться уже привычным запахом кошачьей мочи, который здесь слышен особо отчетливо. Видимо, они все там и живут)., вы попадаете в зону СПА. Тут есть маленькая сауна, паровая комната и хаммам. Температура в сауне кофмортная, а вот в паровой комнате слишком жарко. Все это дело работает до 9 вечера,считаю, что очень рано закрываются! Хотя бы до 10 часов!! Хочется сходить в сауну перед сном, а не сразу после ужина!!! Пляж: пожалуй, это лучшее воспоминание! У Риксоса, кажется, одного из немногих отелей (судя по пустому пляжу) был оборудованный пляж. Приятный персонал (особенно девушка Езгэ(сорри если неправильно имя написала).Очень милая и говорит по-английски. Остальные, спрашивали номер комнаты, но абсолютно не понимали по-английски. Лежаков много, но на пляже и так никого не было. Бар хороший. Я бы отменила систему «Все включено» или хотя бы сделала возможность выбора системы питания! Я бы брала только завтрак, т.к вокруг полно кафе и ресторанов (на набережной и в старом городе), куда мы периодически ходили, т.к. находится в ресторане становилось невыносимо. В итоге, переплатили за систему «Все включено». Считаю, что для городского отеля ( а не пляжного резорта где-то на отшибе) система «Все включено» является лишней и даже неуместной. Сделайте хотя бы альтернативный вариант!!! В общем, ни в Анталью, ни в частности, Риксос мы больше ни ногой!!! Анталья безусловно очень красива, но если такой ужасный сервис и сам отель Риксос(который является одним из лучших судя по отзывам!),то боюсь представить, что происходит в других отелях ниже классом!!! Вообще, конечно в антисанитарией в Анталье что-то нужно делать!!! Как мы заметили, грязно везде и сервис очень на низком уровне! Ну и конечно КОТЫ!! Если Турки действительно так любят котов, как они говорят, то начните хотя бы со стерилизации животных! Остановите это бесконтрольное размножение особенно на территории отелей!!! Запах кошачьей мочи, видимо, будет преследовать меня еще очень долго!!! И вид ободранных кошек на столе, жрущих прямо из ваших тарелок!!!! Больше ни ногой в этот гадюшник!!! Отель оцениваю в 2 звезды! Особенно из-за того, что это Rixos!!!Пока самый позорный отель из всех, что мы посетили!!!!', 'Отель испортился: очереди в ресторане, в номерах воняет кавролином, номера обшарпанные с подкраской местами. Старая сантехника, в барах разбавляют спиртное. Обслуга с недавольными лицами, как будто делают тебе одолжение. Единственный нормальный парень тот, кто минибары обслуживает.', 'Были в отели семьёй с ребенком 8 лет в конце августа 2023. Плюсы отеля: - большой пляж - просторные номера Минусы отеля: - неухоженная территория. Совершенно никто ей не занимается. - везде грязь. Мороженое упавшее в лифте будут убирать несколько дней - посуда в столовой очень грязная. Внимательно проверяйте тарелки, которые берете. - в столовой отеля пахнет туалетом ( особенно в зоне бара) - наша семья подхватила кишечную инфекцию. В целом, отель очень уставший. Кажется, как будто из него хотят выдать последнее не вкладывая денег.', 'Можно было бы много написать про данный отель, но 16.06.2023, играя на отельном пляже в песке с ребенком, муж напоролся ладонью на иглу от использованного шприца. До сих пор в шоке от случившегося и потенциальных последствий. Отель принёс изменения в виде вина и фруктов (что и без того нами было оплачено) и слов успокоения от русскоговорящего менеджера: "ну случилось то, что случилось, что мы можем сделать". Дополнительно проверять качество уборки пляжа никто не вышел. Машин, просеивающих песок, в отеле нет. Берегите себя и детей.', 'Сейчас нахожусь в отеле. Тут есть остатки былой роскоши, ланшафт и большой пляж (каменистый при заходе) Но это не заслуга нынешнего отеля. Сейчас можно констатировать, что это старый ушатанный отель с неисправными номерами и не справляющимся со своей работой некомпетентным менеджментом. Здесь плохо организованно примерно все. Ресепшн. Встретил недружественно. Мы приехали в 7 утра после ночного полета. Я понимаю, что отель не обязан заселять раньше срока, хотя в хороших отелях это делают. Но это был первый турецкий отель, где мне отказали в подключению к вайфай - сказали только после заселения. Если будешь просто сидеть и ждать заселения, тебя не позовут, надо подходить просить неоднократно. При заселении стратегия у них следующая - дать самый плохой неисправный номер, если гость не вернулся, то прекрасно, если вернулся - говорим что номера все заняты, приходите завтра и так далее до изматывания гостя, когда он уже решит, что лучше жить в таком номере, чем ходить к этим на ресепшн. Мне дали первый номер с неисправной раковиной в санузле она была наполнена водой до края и вода не уходила. Невозможно умыться. После долгих препирательств (свободных номеров нет, ага) поменяли на номер в котором раковина работает, но перекосило и не закрываются все двери - в душ, туалет, на террасу (с которой вообще то можно беспрепятственно и в номер зайти, дверь то не закрывается!) Разборка с этим номером продолжается (номеров то свободных нет, ага) Неисправные номера надо закрывать и ремонтировать, а не выдавать гостям! Что такое сервис тут не знают, а на гостей плевать. Ресторан рассчитан не более чем на 50% заполняемость отеля, короче пришли чуть позже открытия - мест нет. Просто физически нет. Поразила неопрятность поваров. Выставленная гнилая черешня. Да много чего неприятного в этой столовке. То же с лежаками их недостаточно, но можно и на песочке полежать пляж то хороший и большой. Это конечно не 5* ни по каким параметрам сервиса.', 'Это отвратительно... Приехали в этот ужас 11 мая 2023 года и пожалели... Еда:неплохая но разнообразия нету Отель полностью грязный все разбитое все обос***о птицами. Номера старые сказать нечего персонал просто молчит. Аквапарк течёт прямо как из ведра. Территория большая... Один вопрос зачем? Там толком ничего нет. Не советую! Совсем не соответствует тому что на фото! 10 дней еле выжили!', 'УЖАСНЫЙ ОТЕЛЬ !!!!! Не ездите сюда ,большая часть положительных отзывов написана персоналом ,полная антисанитария ,в лобби воняет ,персонал хамит а кофе и многие другие напитки с бара просто отвратительные ,на счет интерьера просто молчу ,очень далеко находится ехать 1 час с аэропорта ,персонал в мятой грязной одежде !!! Из хорошего только что быстро заселили и то с этим помогла девушка гид которая везла сюда .Если цените свое время ,нервы и деньги то выберете что нибудь другое', 'Достойный персонал уборщиков.Отстойный вышестоящий персонал.Маленькая столовка.Отвратительная работа кондиционера.Далеко не 5-ка,средняя 3-ка с претензией.Повсюду раскиданы фекалии кошек.Представитель по общественным связям отеля -хамоват и можеть допустить грязные речи необращая внимания на детишек и вообще субардинацию.Выберите более авторитетные и достойные отели,вы не за муравьями и пауками в номере и также не за фекалиями на территории отеля.', 'Второй день находимся в отеле и просто в тихом шоке. Это если вкратце). Во первых, сейчас в Турции температура воды 16-17градусов. При бронировании мы это учли и выбрали номер лагуна делюкс свим апп. Это номер с выходом сразу в бассейн. Это дорогой номер и несколько раз было уточнено подогревается ли бассейн у нашего номера. Туроператор (Sun Fun) и в самой концепции отеля прописано что подогревается круглогодично. При заселении выяснилось, что все отключено, тк в Турции летний сезон))) Холод дикий, но в отеле кондиционеры на обогрев отключены с 1.04)))) Хотите тепла, ловите парочку обогревателей в номер. Тк кондиционер на обогрев выключен и никто его включать не собирается до зимы!!!! Вот такое вам лакшери))) Общении с гестрелейшен это отдельный вид унижения (прикреплю сканы). О чистоте в ресторане судите по фото))) В общем, за эти деньги есть масса реально премиум отелей! При этом цена данного отеля уже сейчас крайне неприличная! Не портите себе отпуск как мы. Единственное что могу добавить из плюсов, спасибо за раннее заселение.', 'Сервис не соответствует заявленной цене В номерах запах канализации В лифте застревают люди В аля карт ресторанах ожидание кофе 20 минут В коридорах на некоторых этажах также пахнет канализацией. Рядом с отелем находится комплекс очистных сооружений и вечером на балконе не посидишь. Единственное, что хорошо в отеле - это основной ресторан, в остальном отель едва дотягивает до 4-х звезд. За эту поездку, мы посетили более 20 отелей по всему миру. Этот я поставил на второе место в анти рейтинге.', 'Добрый день! Проживали в отеле с женой и друзьями с 15-18 сентября 16 числа были на шоппинге, привезли поставили пакеты около кровати. 18 числа в день выезда обнаружили пропажу брендового пакета с вещами внутри. Обратились к управляющему - все что сделали, спросили у уборщицы брала ли она пакет или нет. Она призналась что его выкинула, но по ее словам там ничего не было, хотя пакеты с вещами поставили и не трогали и тем более тогда куда делись вещи. И ничего больше сделать не стали. Сказали езжайте домой мы разберемся за 2-3 дня, прошла неделя - никакой обратной связи! Отель не реагирует никак. Отель не дешевый, а отношение к гостям на уровне мотеля.', 'Всем здравствуйте! 26.07.21 – 09.08.21 (по факту уехал 06.08.2021 об этом моя история!) отдыхал со своим 7-летним сыном в Турции, Анталья отель «Lara Barut Collection» 5*. Я никогда и нигде не пишу отзывов, но та нестандартная ситуация, невольным участником которой мне пришлось стать, не оставляет мне шанса сохранить этот статус) Забегая вперед, скажу, нас с сыном выселили из отеля за несогласие с мнением отеля в лице генерального менеджера по вопросу порчи моего имущества. Ну, поехали: В один из вечеров я с сыном находился в мини-клубе, где мой ребенок играл в футбол с другими детьми. Я стоял с краю площадки у забора, мои друзья были с другой стороны забора. Мы тогда отметили, что слышим запах я извиняюсь Го….на. Но не поняли, откуда он исходит, решив, что произошла какая-то авария, так как после окончания футбола мы меняли локацию в пределах территории отеля несколько раз, но запах преследовал нас повсюду. И только придя в номер, я обнаружил, что авария случилась со мной, вернее с моей брендовой обувью. Я принял все возможные меры, чтобы устранить это самостоятельно, но масштаб бедствия был огромным, обувь была измазана почти полностью, как если наступить в глину после дождя. В течение нескольких последующих дней я пытался решить вопрос с запахом через гест релейшнс, но столкнулся с недовольством той стороны, хотя они все-таки совершили две попытки отхимчистить обувь, результата не было, запах все равно не позволял находиться с этой обувью в одном помещении. Далее они переложили ответственность на меня, никто не извинился за ситуацию. Мой номер и обувь продолжали источать несовместимый с жизнью запах. Мне было неприятно их раздражение и отсутствие каких-либо извинений, я решил, что возможно сотрудники некомпетентны, поэтому обратился к вышестоящему руководству, меня записали на встречу с генеральным менеджером отеля на 06.08.21 в 14:00. Встреча не оправдала моих надежд, передо мной не то что не извинились, так еще и угрожали полицией, услышав, что я фиксирую наши диалоги с отелем на диктофон телефона. После чего я понял, что старая поговорка про рыбу, гниющую с головы, правда в который раз, и смирился с ситуацией, покинув кабинет генерального менеджера униженным (в том числе в глазах своего сына). Примерно через 40 минут мне позвонил гид компании Пегас Саид и сказал, что отель выселяет нас с сыном сегодня. Пегас, конечно, начал подбирать альтернативный отель для нашего переселения, но, думаю, очевидно, что ни меня, ни тем более моего ребенка такой расклад не устроил, наш отдых подходил к концу, оставалось всего три дня (мы брали путевки на 2 недели), сын завел друзей, изучил отель и чувствовал себя значимым в тех условиях. Он в слезах обзвонил новых друзей, рассказав о выселении, и умолял меня вернуться домой сегодня. В общем, переезд был для нас неприемлем по многим причинам, особенно в виду того, что мы ничего плохого не сделали, из-за чего можно было бы выгнать гостя с малолетним ребенком (как уверяют туроператоры, выселение – это крайняя и очень редкая форма взаимодействия с гостями, которой предшествует как минимум пьяный дебош или что-то в этом роде). Я ещё молчу про эпидемиологическую обстановку в мире, которая усложняет в принципе любые перелеты/переезды. Прождав до 18 часов в неведении, куда и как нас переселяют, я собрал чемоданы и покинул Турцию, купив новые билеты (т.к. мои обратные билеты, приобретенные на 9 августа, не подлежали возврату и обмену). Итого, помимо испорченной обуви, испорченного отпуска, зареванного ребенка, испытанного мною унижения, я потерял ещё дополнительно около 35 тысяч рублей на покупку новых билетов, трансфер, а также, как я понимаю, компенсацию за непрожитые дни мне вряд ли возместят. Надеюсь это прочитают владельцы сети отелей Барут и примут меры как минимум в отношении своего менеджмента в вышеупомянутом отеле. В туроператора отправлена досудебная претензия, далее планирую обращение в суд, напишу, чем закончится!)', 'Подошли узнать сколько стоит такси, отправили к носильщикам чемоданов. Увидев что мы только спросить стоимость, разговор с нами не интересен стал… Подошли уже с девушкой с ресепшен и тут началось, что они вызовут такси и нас повезут за 35€ (13км туда) По факту это стоит 15-20$ Отель ведёт себя ужасно по отношению к гостю. Во всех отелях здесь вызывают такси спокойно, на ресепшен с помощью телефона, а здесь скоро через уборщиц будут…', 'Отвратительная кухня, много еды но поесть нечего, даже казалось бы чем можно испортить обычный омлет или яичницу и даже это невкусно от слова совсем. Нет детского меню, детям нечего кушать, картошку фри есть каждый день детям вредно. Официанты не успевают от количества гостей убирать столы, не приносят за стол чай напитки, их сложно отыскать в зале, имею ввиду официантов. Плохая уборка номеров, убирают как следует только после моих замечаний. В номерах нет посуды стеклянной, нет приборов. Детская комната имеет неприятные запахи, отель 5 звезд, по моим убеждениям пахнут должно приятно. Еда в детской комнате каждый день одинаковая, ничего не меняется, салаты кислые и заветренные не обновляются весь день от слова совсем. Ресторан у моря отвратительный, ни одно блюдо невозможно есть, как можно испортить обычные креветки?!!! Почему я в ресторане у моря должна ходить сама с тарелками если я пришла на ужин по Аля карт. У бассейна нет возможности заказать у официанта чтоли бы, нужно коугом ходить самой !', 'Ездили в этот отель с 2000 года , после реконструкции увеличили номерной фонд в 2 раза , территория осталась той же . толпы , : не протолкнуться : антисанитария на шведском столе , общие ложки , июльавгуст жара плюс влажность , пол и купальники не сохнут совсем , на улице плюс 38 , …жуткие условия для выживания , безрассудно ехать в этот отель , особенно с детьми , едва вернулись домой с 40 температурой , номер ресепшен не продлил , хотя говорили что плохо Себя чувствуем , ждали трансфер на улице . , Лежали , от температуры не могли сидеть , номера не обрабатываются , чувствуется зажратость от денег и бронирований , заплатили 380000 руб за 9 ночей с бизнес перелетом туркишами , ужас : сейчас отдыхаем на Крите , жарко , но отлично себя чувствуем и Турцию вспоминаем в страшном ночном кошмаре .', 'Останавливаюсь в этом отеле уже третий раз и впервые решила посетить утреннюю встречу с гидом. Гид Абдула сначала был очень милым, остроумно шутил и просил всех записывать всю информацию, чтобы не забыть (даже выдал для этого листочки с картинками об экскурсиях и ручки). Я первый раз в жизни решила записать все что он предлагал (точное время экскурсий и стоимость!!!!). Очень понравилось его предложение поехать на день в Стамбул и естественно записала точное время отъезда из отеля и возвращение в отель: 6.30-22.30. Мои соседки на встрече несколько раз переспросили меня во сколько уезжаем из отеля и во сколько возвращаемся и я им показала свои записи. После оплаты экскурсии 199$ за человека (а я была с дочкой, те 398$ за двоих) оказалось, что мы должны выезжать из отеля в 3.00!!!! Утра!!!! Я подумала, что он ошибся, тк записала его слова на его же листочке!!! Но нет! Он сказал, что он имел ввиду, что мы вылетаем в 6.30, а не выезжаем из отеля!!! Я в шоке! Я думала я в 5-звездочном отеле, а оказалось, что я на турецком рынке и то, что я записала с его же слов ничего не значит. Я естественно отказалась от этой экскурсии, тк меня ввели в заблуждение рекламой, но мне отказались вернуть деньги!!! Никогда не ходите на встречи с гидом Абдуллой!!! Надеюсь завтра переговорить с руководством отеля об этом неуважении к постоянным гостям отеля и вернуть свои деньги! Напишу о результатах в следующем посте.', 'Посетили Турцию в первый раз и в первый раз по " все включено". Реишили посмотреть что это такое... Всегда беоем в разных странах гостиницы не меньше 5*. Взяли гостиницу через Пегас Туристик. Из плюсов только линейный персонал на ресепшене и менеджер Халит по турам. Минусы: Не пейте коктейли, бадяжат все подрят, что вкус получается помоев, лучше пить в чистом виде алкоголь, пиво, вино, мартини и так далее. Еда на 3. ни разу не поменяли постельное белье... ребят, ВЫ заявляете себя 5* гостиницей, и я не ОБЯЗАНА звонить каждый раз и говорить вам ,что нужно сделать!!! Вещи на стирку, три добивались чтоб забрали! Не надо говорить, что у вас большая загруженность гостиницы и персонал не справляется, если заявили себя 5* то и соответствуйте этим звездам, а не ищите оправдания! При выезде выяснилось, что еще и приписали лишний комплект одежды, за который надо заплатить, начали выяснять, то же самое бла бла бла! В итоге мы попросили посмотреть камеры, технично весь персонал разбежался, сказал что займет два-три часа и так и никто не подошел!👎🏻 Менеджер прачечной так вообще орал как на базаре! в первый раз такое вижу! Не гостиница, а сарай базарный, раз управляющие позволяют своему персоналу так себя себя к гостям! В общем не знаю что было в этой гостинице раньше, но сейчас это ужас... Гостиница все подпортила впечатление о Турции. Может и вернемся в эту страну (но только не этот отель точно!) лет так через ..цать, но точно не в ближайшие года ,даже ради "все включено".', 'Мы все ещё находимся в этой гостиницу, сегодня 2 день из 7 и уже хочется поделиться отзывом, чтобы читающие приняли решение - приезжать сюда или нет. Начнем по порядку Все началось с приезда. Входная дверь была так ужасно покрашена, что появилось впечатление, что всё в гостинице будет в таком духе. Я решился сюда приехать только потому, что бабушка сюда уже приезжает более 12 раз. Мы сняли 3 номера. Бабушке и дедушке дали номер кажется в самом конце здания, куда идти от лифта до 5 минут пешком. А им лет за 70 и никого это не волнует. Количество лифтов всего 2, которыми в основном пользуются все постояльцы. Учитывая что здесь почти каждая семья с детьми и когда они заходят в лифты с коляски, просто нет места для других. Количество самих лифтом критически недостаточно для такой загруженности отеля. Во всей гостинице царит ужасная жара. Когда только зашли в лобби, я просто был поражен температурой в помещении. На мое удивление сотрудник сказал, тут везде так. Его слова были искренними, так как в первый же день мы решали вопрос толком неработающего кондиционера в номере. Номер. Ужасная шумоизоляция, стены из гипсокартона, крошечный балкон на 1 человека, и крошечный санузел. Мебель в номере надо утилизировать. Кровать такая мягкая, что боли в спине обеспечены. Я такие кровати видел может быть лет 10 назад в последний раз. Кондиционер еле тянет, чтобы мало мальски охладить комнату. В комнате конечно царит все +27 и выше, но хоть на этом спасибо. Хотя видно, что в комнате недавно делали косметический ремонт, но явно бросается в глаза то, что мастера явно были наняты из за низкой стоимости работы. Тяп ляп можно не искать, он повсюду. А кресло, которое уже черное, где должны быть белые элементы, даже не хочется комментировать и тем более садится туда. Уборка такая же как и все остальное, даже мусор не забрали и пепельницу не почистили/ Ресторан. Там просто муравейник, бестолкоковая еда и жара. Каждый раз приходится искать место для посадки. Ресторан просто не рассчитан на такое количество постояльцев. Чтобы взять еду, надо пройти через лабиринт. Мясо готовится на улице, и в пекло приходится идти туда и брать мясо. Никакой логической расстоновки блюд нет. Ходишь змейкой и набираешь себе еду. Качество блюд сильно хромает. Мясо кушать невозможно, например сегодня я взял шашлык, который на вид был хороший. Оказалось что его невозможно кушать, состоит из жилок и очень жесткое мясо. На первый взгляд кажется что еды очень много, но реально вкусно покушать мне не удалось ни разу. Везде овощи и супы, а мясо просто не вкусное. К тому же, все что мне более менее понравилось, за этим блюдом выстраивается очередь. Как бы смешно не казалось, в таком отеле самым вкусным блюдом оказался гамбургер. Да, именно гамбургер, который выдается как снэк в любой другой гостинице. Даже не сам гамбургер, а котлета. Про разбитые тарелки, где выдают уже готовые блюда я промолчу, но фотки загружу. Море и бассейн. После обеда море это ил с водой. Думаю в этом гостиница не виновата, просто местность такая. Поэтому если Вы сюда даже приедете, старайтесь выбирать утреннее время для моря. Так как после обеда стабильно волны, очень не комфортные. Сам пляж очень грязный, лежаки в песке, как я понял, единоразовая уборка происходит в конце дня. Количество бычков на пляже удивит любого. Одним словом, на пляже нет уборки. Лежаки, это просто аут, они самые дешёвые которые есть в общественных пляжах европы. Гостиница которая установила не маленькую стоимость за проживание и с 5 звёздами (я бы дал максимум 4 звёзды с натяжкой), просто вынуждена предоставить лежаки хотя бы среднего качество с наполнителем, а не эти пластмассово тканевые лежаки. Это пол беды. Потому что лежак Вы возле бассейна просто не найдете. Количество гостей явно превышает возможности отеля. Из обхода 6 бассейнов нам удалось найти всего 2 лежака на 4 человека. Это не укладывается в голове, но это факт. Культура отдыхающих/ Был удивлен культурой отдыхающий людей в этом отеле. Крики, шумные разговоры тут в порядке вещей. В номере из за и так плохой звукоизоляции, Вы будете слышать все что происходит по соседству и в коридоре. Если Вами занятый лежак, в виде оставленной одежды, не полотенца, займут другие постояльцы, не удивляйтесь. Наша одежда и полотенца просто оказались на траве, недалеко от занятого нами лежака. На вопрос, Вы ли убрали нашу одежду туда, в наглую говорят, нет, мы ничего не трогали. Я не буду говорить за всех, были и адекватные гости, но гости у кого явно низкая культура поведения присутствует, их более чем достаточно. Личные границы не соблюдаются, могут встать впритык в очереди за блюдом. Отель точно с уклоном для детей, все таки отель себя презентует как kids concept? Однозначно нет. Единственно логическая связь с этим концептом, это количество детей в отеле. К сожалению, отель даже не сделал тент над детской площадкой, где печет солнце и соответственно ни одного ребенка там нет. Более того, в отеле имеется бассейн 16+. Однако это просто надпись, на деле никто не контролирует. Это один из таких же других бассейнов, где купаются дети. Поэтому я тоже плюнул на их неработающее правило, и с дочкой начали плавать в этом бассейне. Вывод: отель своих денег не стоит однозначно. Его надо закрыть и приводить в чувство, на ковроланы даже смотреть больно. Все настолько старое. Касательно и количество звезд, на мой взгляд, это еле дотягивающая гостиница до 4 звёзд. Кажется отель просто делает деньги из последнего вздоха гостиницы. Мне не нужны оправдания или извинения от гостиницы в виде ответа на комментарий. Просто делайте свою работу как полагается. Я все равно больше к Вам не приеду. Если что то хотите обсудить лично, мой номер 1029.', 'Мой отзыв для того, чтобы люди покупая путевки по отнюдь недешевым ценам в этот отель понимали, что они не получат совершенно ничего за это! Питание-это кошмар, на завтрак яйца во всех видах и булки, на ужин -картошка, макароны, причем макароны даже без добавок, если вы хотите сделать пасту, то вам предложат красный и белый соус, все! Мяса в отеле нет , рыбы и морепродуктов тоже! Мы пытались жаловаться, отдыхаем в Турции подряд уже лет 10 и такого мы не встречали, жаль не могу загрузить видео , мясо( в основном курица) максимум один лоток на весь ресторан и замешана в овощах или макаронах так, что достанется это мясо точно не всем, его там надо искать, была за 4 ужина 2 раза на гриле рыба, все! Кальмаров нет, мидий нет, с питанием просто Ужасно! На фото прикрепила Аля карт блюда! За этот салат от шефа они берут деньги 😅🙈ну и все там такого вида и вкуса, рис от шефа тоже представлен 🤦‍♀️Позор! Отдыхаем с детьми и друзьями, все расстроены, отдых испорчен, персонал отеля совершенно не знает языков, я не говорю про русский, но и с английским беда полная, не допроситься принести напитки, хочешь пить, иди и возьми сам! Бар отвратительного качества, пишут, что входят импортные напитки, по факту-нет! Вместо текилы наливают из-под стола какую-то муть, когда контролируешь, льют хотя бы Ольмеку, все нормальные напитки экстра! Отель и Пегас предложили нам оплатить штраф и переселиться в другой отель, за что тоже должны оплатить 2000 евро! В общем писать могу много, но напишу одно, если вы на распутье, вычеркивайте этот отель из списка, он не стоит и 4 звезд! Все очень плохо, только лишь территория хорошая! Персонал отеля указывает, что на сайте отеля никто не обещал шампанского и сервиса хорошего 😅Не подкопаться, так и есть! А еще почитайте отзывы плохие, готова подписаться под каждым словом! Некоторые люди год ждут отпуска, откладывают деньги, чтобы потом есть тушеную капусту и пить паленый алкоголь? Нирвана космополитен за те же деньги в разы качественнее! Мы еще находимся в этом отеле и даже сотрудники нам говорят, что отель экономит сильно 🙈', 'Это позор. Только что приехал из замечательного отеля Royal wings в этот сарай. Заселили в номер с видом на свалку. Полы в номере грязные. Стены и поверхности все заляпанные. Заменили мне номер. Во втором номере ситуация такая же. Подушки - скомканная вата. Балкон - улица, горничные не считают, что он требует уборки. Пластиковая мебель самая дешевая. Заявленный минибар пуст и содержит грязные стаканы от предыдущих гостей. Номера требуют капитального ремонта!!!! Бедные сотрудники отеля. Это не их вина. Владельцы экономят на всем. Персонала не хватает. Заехала конференция из Казахстана, отель битком. Бедные горничные! Они работают за три копейки, а руководство сети миллионами гребет бабло, не заботясь о комфорте гостей. В ресторане за ужином нет свободных столов. Гости в шоке. К раздаточной зоне не подойти. Ценник на лето конский. Я очень надеюсь, что менеджмент отеля задумается над проблемами и наконец-то проведет в отеле реконструкцию, или хотя бы отмоет его от грязи. Это позор.', 'Очень разочарованна отелем!!!! Позиционируют себя как хорошая 5, а по факту помойка. Номер стандартный, ремонт более менее, но в ванной плесень. Уборка плохая, только для видимости, даже пыль с тумбочек не протирали. Ресторан!!!! Посуда грязная, чтобы найти чистый стакан нужно потрудиться, те которые чистые, в каком-то налёте, есть не приятно. Пляжные туалеты тоже очень грязные, во второй половине дня, лучше вообще туда не ходить…. Пляж понравился- хороший, широкий, но опять же не убранный, стаканчики, окурки, палочки от мороженного- это всё в изобилии. Территория большая, красивая. Аквапарк понравился. И вишенкой на торте стали съеденные личинки в сушёных апельсинах на ресепшене ими украшают напитки, когда встречают гостей!!! БОЛЬШЕ СЮДА НИКОГДА!', 'Рейс перенесли . сутки в отеле пропали. приехав в отель нас спросили , почему вчера не приехали. Мы ответили рейс перенсли на сутки туда и обратно. Можно ли сместить на сутки вперед так как задержка была и мы не получили услугу. Ответили категорично нет. Платите 300 евро за сутки. Сдвигать ни чего не будем. Неприятно. Могли бы встать в положение. мы же просили просто передвинуть проживание. так как оплатили 12 дней и не просили большего', 'На ресепшене на тебя даже глаза не поднимают, видно, что не рады. Вечером при выходе из душа сломалась ручка. 1,5 часа во влажном душе. Система оповещения не работает. На стук никто не приходит. Извинений ноль. Жена отравилась в столовой.', 'Отдыхали с 07.10.2022 по 14.10.2022 .комната 234 . Надо было обождать заселения , выдали ключ от номера и ждали свой багаж , но не дождались , пришлось самому чемодан тащить - 5 звезд заведение . На пляже в баре нету даже самого простого коктеля -5 звезд. В центральном баре Есть коктели-нонэйм-бурда --5 звезд . Питание от супа (клейстер . томатная паста) до выжаренной рыбы и все с томатной пастой -5 звезд .Не было одеял , только тонкое покрывало -5звезд ! Басейн ооченьхолодный и единственная горка неработала.И вишенка на торте , После перестрелки мы боялись выйти из номера и последние три дня даже уборщицу не пускали . Это заведение тянит на 2-две звезды, не больше.', 'Отдыхали в отеле с 9.10. 22 по 13.10.22. 12.10.22 в 8 часов вечера перед дверьми отеля , под козырьком главного входа состоялась стрельба, расстреляли в упор машину владельца или менеджера отеля. В это время там же находилась я и другие гости. Человек с большим оружием в руках шёл на нас и мы думали, что это теракт. Мы не сразу поняли, что это внутренние разбирательства, я бежала, слыша звуки выстрелов и в ожидании, что мне будут стрелять в спину. После проишествия нам, гостям отеля не были принесены извинения, нас не опросили, как свидетелей, пришлось на следующий день самой вызывать полицию. Отель занял очень недружелюбную позицию, пытаясь всячески замять ситуацию и давя на нас. Я настоятельно не рекоммендую никому приезжать сюда, так как именно в этом отеле ваша жизнь подвергается опасности, а руководство пытается всё скрыть .', 'Была в отеле Порто Белло с 25 по 30 сентября. Приехала через туроператора Великолепный век. Я сама с Кыргызстана. Хочу отметить отвратительное отношение персонала к проживающим в отеле. При приезде дают браслет (для Средней Азии оранжевый, для РФ голубой и для Европы вроде череый). В день экскурсии нас привезли в отель только ближе к 3 часам дня. Обед уже закончился. Нам сказали идти в снек бар. Когда я хотела взять меню, мне даже его не дали в руки, сказав что нельзя, чтобы я села и ждала шаурму которую они приготовят!!! Шаурма была ужасная на вкус, это даже шаурмой не назовешь. Когда я пошла к гиду Халиде, которая постоянно находится на 0 этаже, сказать что как такое возможно, хотя у меня размещение в отеле было по системе "Все включено", на что она включила "дурочку " и сказала "да как такое возможно, я разберусь". Естесственно это все так и осталось. Также мороженое хорошего качества дают только европейцам, а остальным дают дешевку на пляже. То же самое когда я подошла в ресепшн, мне сказали что поговорят и выяснят почему так. Ручка от двери все время заедала и мне приходилось по несколько минут пытаться открыть дверь. Я позвонила на ресепшн и попросила чтобы исправили. Обещали исправить, но конечно никто этого не сделал. Еда однообразно безвкусная. Пляж из камней, очень больно ходить. Номера убирают плохо, так еще и украли ремень. Единственная водная горка имеющаяся на территории и то была закрыта. Вода в бассейне холодная. В крытом бассейне сказали подогрев подключат к концу сентября, но и этого тоже не было. Одним словом ооочень любят они обманывать людей. Не советую вообще ехать в этот отель!!!', 'Если вы едете не с группой а самостоятельно, отель категорически не соответствует цена - качество. Не рекомендую. Грязные номера, в спа пахнет сыростью, не работает кондиционер. В общем, не вернёмся сюда больше.', 'Читала про прокуренные номера, но чтобы настолько. Останавливалась на 2 ночи и это самое ужасные ночи. В номере ужасный прокуренный запах и он не выветривается.. Откуда он источается .. То ли им пропитано всё.. То ли с системы кондиционирования затягивает.. (в отеле цент. кондиционер)Но спать в кровати невозможно.. Простот задыхаешься от этого запаха. Пожаловались на ресепшн, но максимум, что они сделали, то распылили в номере что-то, но проблему это не решило.. В итоге в номере не находились.. Ночью с открытым балконом в + 15 не очень. Не спали в отеле, вещи хранили на балконе.. Вообще с такой проблемой столкнулись первый раз, уж сколько отелей за плечами.. Но чтобы невозможно дышать в номере - это за гранью']

In [4]:
test_reviews = ['Очень комфортный отель, удобное расположение в центре города. Превосходный галечный пляж, и это в городе! Особо порадовал футбольный детский клуб, занятия в котором проводили русскоязычные тренеры шесть дней в неделю, а такое комплект футбольной формы Rixos Downtown Antalya, который ребёнок получил в подарок от отеля😀 В отеле есть хорошо оборудованный тренажёрный зал. В Rixos Downtown Antalya очень приветливый персонал! И, конечно же, качественное, разнообразное питание. Нам очень понравилось 😉', ',самая ужасная орущая анимация за весь мой отдых. При этом администрация отеля никак не реагирует. Орёт псевдо Таракан какие-то ужасные поппури, барабаны долбят до 2 ночи. Просто треш. И даже днем тоже самое. Видимо для глухих , слышно этот ужас на всем побережье. Наслаждения мало не спать ночами заплатив немалую сумму. Не рекомендую!!!@', 'Пишу отзыв, находясь в лобби отеля и дожидаясь трансфер в аэропорт. Так что отзыв будет по горячим следам. Долго выбирали отель, основываясь в основном на отзывах туристов и не прогадали. Отель рассчитан на семейный отдых. Много туристов с детьми и тех кому 40+. Отель находиться удалённо от города Серик (20км), поэтому добраться до города проблема, только такси. В этом есть свои плюсы и минусы. Архитектура отдельного комплекса очень интересна, много разных локаций, которые сразу и не обнаружишь. Настоящий квест! Нет ощущения переполненности отеля. Теперь обо всем по порядку. Приехали в отель в девять утра. На ресепшен, на просьбу заселить сразу, отреагировали положительно. Номер предоставили с прямым видом на море. Номер шикарный, большой просторный. Всё работает, все есть в наличии. Мини бар пополняется ежедневно. (кому интересно номер 2218). Территория просто огромная, вся в зелени. Извилистые дорожки потерритории доют возможность погулять и уедениться, особенно в вечернее время. Пляж: огромная полоса песчаного пляжа отеля в одну и другую сторону переходит в дикий пляж. Это даёт возможность прогуляться по прибрежной полосе. Проблем с лежаками никогда не возникало. Отдельная тема - питание. Это просто бомба!!!!! За шесть визитов в Турцию в разные отели такое обилие и разнообразие блюд я встречаю впервые. (супруга на ужин насчитала 56 видов пирожных!!!!!). Про алкоголь: пиво довольно таки приличного качества, неплохое вино. Советую посетить рыбный ресторан а-ля-карт, с видом на гольф поля. Рыба дорадо просто объедение. Огромная благодарность девочкам с guest relation (в особенности Евгении), всегда идут на помощь, любую просьбу решают на раз, два, три. Теперь немного ложки дегтя: Отельная команда аниматоров оставляет желать лучшего, особенно вечерами. Мало конкурсов, не заводит отдыхающих, не клиентоориентированы. На вечерней дискотеке отдыхающие не танцуют. Но приглашённые артисты сглаживают этот недостаток. В целом отель превзошёл ожидания. С удовольствием вернёмся сюда в следующем году и будем рекомендовать к посещению своим друзьям и блюзким!', 'Мы были с 9 по 16 сентября.Все супер : и номер, который каждый день убирала Фатма,приятная и улыбчивая.Изобилие блюд ,десертов на любой вкус.Пляж песчаный, аквапарк и детская площадка. Особенно хочется отметить аниматора Женю, девочек наших невозможно было увести от нее,так она их увлекла,за что ей наша благодарность,мы могли спокойно отдыхать.Все было превосходно и хочется сюда вернуться вновь.', 'В Турции отдыхаем каждый год и любим посещать пару отелей за одну поездку! Ездим с семьей с 3 детьми😊 Давно хотела попасть в этот Отель,забронировали его на 3 ночи связавшись с менеджерами отеля по телефону. Бронировали номер делюкс супериор.большое спасибо за предоставление семейного двухкомнатного номера с перегородкой,о чем не просили, а увидели уже при заезде в номер❤️ Номер комфортный ,но боюсь что для семьи двое с двумя подростками будет тесноват,мы долго размешали свои чемоданы, которые мешали честно говоря.но это не испортило отдыха нисколько! Заселили в корпус с видом на небольшой бассейн-тихий и приятный на первом этаже ❤️ Пошли погулять По территории-везде чистота,у бассейна была движуха-пенная вечеринка,все все шезлонги были заняты ! И кстати они всегда были заняты в течении дня,а вот на пляже было очень свободно и всегда находили место ,шезлонги находятся на хорошем расстоянии друг от друга,песочек вокруг,вход в море галька ,далее снова песок. Море на удивление тёплое было под конец октября и купались каждый день Питание: были в разных отелях,но здесь на удивление безумно приятная кухня,моя любовь -баклажанчики в кисло сладком соусе,много мяса разных видов,сыры -такого ассортимента я не видела нигде, сладостей полно ,но я была к ним равнодушна😄.Очень удобно что «кухня» разделена на 2 зала И главное что всегда без проблем находили места! Какие умнички и пчелки работники зала-махом убирали столы и освобождали для вновь приводящих.Для малышей детские стульчики ,всегда чистые и «одетые в полиэтилен» Печаль только с мороженым,оно правда невкусное и порошковое ,дают только на пляже в «светлое время».. Детей полно и с этим нужно что то делать,как же Турция и без хорошего мороженого.Нам после сети отелей Kamelya Collection ,где порядка 10 видов в пачках и вкуснейшего Algida в шариках -здесь оно показалось совсем скудным.. На пляже был ресторанчик ,где с удовольствием обедали,притом главный ресторан работает только на завтрак и ужин.Выбор на пляже небольшой,много фастфуда -но в целом все вкусно. Вот с рыбой тоже печально-вкусной не было совсем,хотя очень хотелось 🐟 Алкоголь: хоть я не любитель крепкого ,но кому очень важно -в любом баре он импортный,все что душе угодно -коктейли мохито и пино колада ,апероль и другие такие ,как положено! Мужчины пили Вискарь и Егермайстер,все доступно и не из под полы как в некоторых отелях! Развлечения: Большой Аквапарк успели посетить один раз,горки хорошие,народу не много.А в детский ходили каждый день.Также детям очень нравилась детская комплекта в здании с горочками,лабиринтами,каруселькой в виде акул,бассейна с шариками-все немного потрёпанное и пора бы немного реанимировать ,но мои дети обожали это место💖 В самом детском клубе не были ни на одном мероприятии но видели что деткам нравится Вечерние шоу:попали на октоберфест,понравилось,но муж сказал -какой октоберфест с тем же самым пивом эфес ,что пьём в течении дня,хотелось бы новенького на такой случай и желательно темного или бутылочного (учтите на следующий год!) На вечерних шоу не были ,единственное однажды на «живом концерте» на сцене подслушали группу девушек,очень здорово они пели! 3 дня пролетели незаметно и мне не хватило! Несмотря на мини нюансы этот Отель один из лучших на равне с любимыми Spice Hotel и Kamelya Collection. С удовольствием приедем к вам в следующем сезоне на подольше,благодаря чистоте ,гостеприимству,безумно вкусной кухне,детском комнате и отношению всего персонала-начиная от уборщиц ,заканчивая барменами ! Вспоминаю вас с улыбкой и уже скучаю', 'всем доброго времени суток, отдыхом в этом отеле в сентябре! признаем уже не первый раз, все нравится, всем рекомендуем. анимация хорошая, живая музыка на пулбаре классная, еда вкусная! персонал очень добрый и дружелюбный!', 'Отвратительный сервис и обслуживание. Дали номер со сломанным кондиционером и никто из персонала не реагирует на проблему. Заселились пожилые люди и в жару 35 градусов вынуждены проживать в таких условиях. Номер менять отказались, деньги возвращать тоже отказались. Якобы «приходил» техник и сказал, что все работает - по факту проблема осталась.', 'В Турции 4-ый раз. Этот отель самый худший из всех. В номере очень шумно до 2 часов ночи, спать невозможно. Номер очень маленький и грязный. В ванной комнате воняет канализацией. Свет в номере моргает даже ночью при всем выключенном освещении. Этот отель точно не 5 звезд, даже 4 звезды с трудом. Единственное что порадовало это более менее разнообразная еда.', 'Абсолютно разочарована отелем. Обслуживание официантов никакое( приходилось самой искать столовые приборы, убирать грязные тарелки, наливать чай). И даже если дадите чаевые, то ничего не поменяется... Номер попался чистый, но если не оставить чаевые, то постельное белье меняться не будет... Цена отлеля завыша, больше не вернусь.', 'Администратор отеля не помог заселится ,по базе были ошибки .Заехали только в 16:30 🤦🏻‍♂️ Попросили нас подождать около 30 мин . Постели были грязные в зеркало сломанная . Администрацию оцениваю шкале 1 ,а персонала молодцы 5+.Фалкон 1511 бутик отель и Фалкон отель администраторы не смогли не как решить нашу проблему !']

In [5]:
all_pos_words = set() # создаём множества, куда соберём встреченные в отзывах слова
all_neg_words = set()
only_pos_words = set() # создаём множества, где будут содержаться уникальные слова (только в положительных текстах или только в отрицательных)
only_neg_words = set()

In [6]:
def collect_words(reviews, words_set): # функция, записывающая слова из списка отзывов в начальной форме в нужное множество слов
    for text in reviews:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha():
                parsed_word = morph.parse(word)[0]
                words_set.add(parsed_word.normal_form)

In [7]:
collect_words(positive_reviews, all_pos_words) # собираем слова из положительных и отрицательных отзывов
collect_words(negative_reviews, all_neg_words)

In [8]:
not_neaded_words = all_pos_words & all_neg_words # пересечение множеств слов - слова встречающиеся и там, и там
only_pos_words = all_pos_words ^ not_neaded_words # слова из положительных отзывов, которых нет в пересечении
only_neg_words = all_neg_words ^ not_neaded_words # слова из отрицательных отзывов, которых нет в пересечении

In [9]:
def determine_tonality(tst_review): # определяем тональность отзыва
    test_list = [tst_review]
    test_set = set()
    collect_words(test_list, test_set) # собираем список слов из отзыва, используя функцию, созданную выше
    pos_counter = 0
    neg_counter = 0
    for word1 in test_set: # для каждого слова проверяем, есть ли оно в каком-либо из списков уникальных слов
        if word1 in only_neg_words:
            neg_counter = neg_counter + 1
        if word1 in only_pos_words:
            pos_counter = pos_counter + 1
    if neg_counter > pos_counter: # каких слов больше, такая и тональность
        return("neg")
    elif pos_counter > neg_counter:
        return("pos")
    else:
        return("neutral")

In [10]:
right_answers = ['pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg', 'neg', 'neg', 'neg'] # список правильных ответов для тестовых отзывов
answers = []
for rev in test_reviews: # для каждого тестового отзыва определяем его тональность
    answers.append(determine_tonality(rev))

In [11]:
import numpy  # считаем accuracy
import sklearn.metrics

r = sklearn.metrics.confusion_matrix(right_answers, answers)
r = numpy.flip(r)

acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.6


In [12]:
print(len(only_neg_words))
print(len(only_pos_words))

1283
391


Так как негативные отзывы обычно длиннее положительных, у нас получилось сильно больше слов, которые мы определили как негативные. Логичным шагом для улучшения работы алгоритма будет увеличить число положительных отзывов.

In [13]:
positive_reviews2 = ['Самый лучший отель. Все на высшем уровне начиная от персонала заканчивая кухней. Еда очень вкусная и разнообразная. Персонал очень дружелюбный, особенно менеджер Сабина. Комнаты очень чистые. Будем в Falcon отеле снова и снова.', 'Здравствуйте мне и моим друзьям очень понравился отель. Мы были в восторге от увиденного и ещё нам очень понравилось обслуживание в этом отеле отдельную благодарность хочу сказать Уста Ахмед оглы Вайде она была очень внимательна ко всем моим друзьям спасибо ей за такую внимательность!!!', 'мы в восторге, отдохнули на ура! очень добрые и вежливые работники отеля😍 все всегда чисто и свежо 🙏🏻 советую всем побывать тут хотя бы раз , у них еще есть аквапарк, мы тоже обязательно вернемся в следующем году 🎆', 'Остался очень доволен отелем и персоналом. Очень большой выбор вкусной и разнообразной еды. Приветливый и отзывчивый коллектив. Отличный пляж и особенно понравился дайвинг. Отличный диско бар с живой музыкой и зазличными шоу программами. Я остался доволен. Рекомендую особенно для семейного отдыха.', 'Хочу выразить благодарность персоналу отеля, особенно Ваиде, за внимательное и радушное отношение к нам. Очень хорошая кухня, замечательный вид из отеля и музыкальные вечера.Советую посетить, приедем еще и не раз.', 'Отель моей любимой сети IHG в Анталье. Находится немного в стороне от Старого города, в курортной зоне. В этом для меня есть плюс, ведь вдоль моря идет продолжительная набережная, по которой я с удовольствием совершал утренние пробежки. Номер у меня был с балконом и видом на море. При отеле есть бассейн, небольшой тренажерный зал, спа и удобная парковка.', 'Превосходный отель, прекрасное расположение. Обслуживание на высшем уровне. Приятный отзывчивый персонал. Отдельная благодарность гест релейшн менеджеру Рим, профессионал своего дела, а также большое спасибо Эрхану и Еролу за гостеприимство.', 'Хочу поблагодарить весь коллектив спа в отеле и особенно массажистку Гульсину за отличную работу. Искренне рекомендую 🤗 замечательный специалист и чуткий человек. Относиться к своей работе с душой и отдачей. Гульсина желаю вам профессиональных успехов и всего наилучшего 🥰', 'Остались с братом на одну ночь. Все супер, команда отеля на высоте! Особенно работник Рецепции. Хотели ещё продлить но к сожалению места были все забронированы . Рум сервис очень быстрый, завтрак был на вкусный! В следующий раз останусь на дольше) спасибо за приятный отдых!', 'Это наше первое посещение Турции с подругой, и сразу в самое сердце!) Мне говорили, что я влюблюсь в Турцию, когда там побываю - так и произошло!!!))) Отдыхали в Crown Plaza - отличный отель с очень вежливым персоналом, хорошие комнаты, есть своя спа-зона, хороший бассейн и очень вкусные завтраки)) отдельное спасибо Hatica, Ozan, Mustafa, Guest Alper and Rim - очень доброжелательные и отзывчивые сотрудники отеля!) Спасибо за ваше гостеприимство, благодаря вам я хочу вернуться сюда ещё раз))', 'Еда- отличная , обслуга дружелюбная, никто не пристает, официанты- обучены- идиотов среди персонала - не заметил, бассейн и море- присмотрены , Единственное- бывают свадьбы после 19 и персонал демонстративно - заранее начинает убираться около бассейна, тем самым мешают отдыхать и купаться, хоть время отдыха около бассейна в гостинице обозначено до 19 часов', 'Ездил один в августе 2023 на 5 ночей. отель очень понравился. отличный просторный номер. светлый, солнечный, заходить в него было одно удовольствие. ванна-душ отлично. завтраки шведский стол, в принципе все стандартно. много фруктов - оливок. Обеды и ужины в первый день показались скромными (одна курица), но потом все изменилось и были и телятина и баранина. Готовят очень вкусно. море - сразу на глубину ныряешь и тут немного раздражают люди, не умеющие плавать, которые держатся за лестницу. но это не к отелю претензия (хотя можно сделать параллельно морю от лестницы перила и за них бы все держались). Персонал улыбчивый и доброжелательный. С супругой поеду обязательно еще. а с детьми - нет, без песка они заскучают.', 'Отдыхали с мужем. Хороший отель на 1 линии с великолепным видом на море и горы. Хорошее обслуживание, очень гостеприимный и отзывчивый персонал. Если есть проблемы с языком, то их решит русскоговорящая девушка Хусния на рецепшн. Еда в ресторане разнообразная и вкусная, ежедневно добавляются новые блюда. Шеф-повар знаток своего дела. Очень вкусные пирожные и десерты. Замечательное горячее, много зелени. Хотелось бы большего ассортимента фруктов. Днем вкусные гамбургеры у бассейна. Номер убирают хорошо, регулярно пополняют гель для душа, шампунь, лосьон для тела. В номере есть приятные мелочи (тапки, набор ниток/иголок, ложка для обуви, расческа и т.д). Неудобно, что только одно одеяло. но проблему решили быстро. У бассейна и на пляже есть бар и никаких проблем с напитками. В отеле нет нескончаемых вечеринок, нет пьяных, приставучих зазывал. Замечательное место для любителей спокойного и уединённого отдыха. Большое желание вернуться туда еще раз! Просьба решить проблему с магнитными ключами, т.к. с одним ключом двум гостям неудобно', 'Прекрасный СПА и дружелюбный персонал! Хочу отметить работу массажистки Сельмы и Вейсель бея, который делал пилинг в хамаме. Спасибо! Кроме процедур можно поплавать в подогреваемом бассейне. Обязательно вернемся! Очень рекомендую!', 'Благодарю отель Адонис, супер сервис и комфорт, всё очень чисто и приятно, всегда на высоте. Очень вежливый персонал и менеджеры отеля всегда гостеприимные и отзывчивые люди. Очень рад у вас отдыхать 😀😘💯🍾👑👑🤠', 'Отдыхали в сентябре в этом отеле . Ооочень понравился !Вид с балкона восхитительный, по дороге на пляж еще лучше!идешь и наслаждаешься очень озелененными аллеями, хочется гулять бесконечно. Пляж галечный , вода прозрачная, купили кораллки для комфортного захода. Вода Средиземного моря держит тело на поверхности и мне, плохо плавающей, не страшно было заплывать вглубь! Море восхитительное! Мы прошли все достопримечательности в Анталии, вправо от отеля 30 минут до старого города, ворот Адрианы , красивый порт , скалы. Влево макдональдс, океанариум, тц Мигросс. Отель твердая 5. Очень зелено, обеды ужины завтраки проходят на улице, восхищение и наслаждение от свежего морского воздуха , окружающей листвы. Котики не надоедали , мы их подкармливали мясом, колбасой. Отель не уставший совсем, как пишут в отзывах, мы начитались и думали нас ждут руины. Чисто , уютно , неубранной посуды не видела. Отель большой , три лифта ездят исправно и быстро. В номере очень уютно и все в идеальном состоянии. Удобные матрасы , подушки. Постель меняли раз в три дня , полотенца и уборка каждый день. Претензий нет. Еда 3,5. Вегетарианцы будут счастливы. Еды полно, наедались и переедали, но мясо и рыба приготовлены невкусно , и на гриле тоже не понравилось , какой то особый у них маринад. Очень вкусная пицца в баре у бассейна, сочная много сыра, тесто мягкое!и луковые кольца- хрустящие и очень вкусные. Также на обед и ужин на улице готовили вкусные турецкие лепешки с разными начинками !Восхитительно! Это было самое лучшее блюдо в отеле! Приедем ли мы сюда еще? Да!!! Море прекрасное! Виды , захватывает дух! Есть куда прогуляться! Пиво очень вкусное!', 'Отель очень понравился Ездила отдыхать в отель в июне 2023 года. Отель спокойный, больше подойдет для семей без детей. Заселение в данном отеле раньше 14:00 не предусмотрено. Немного тяжело, если у тебя рейс утренний. В 7:00 я была уже в отеле. Но тем не менее персонал который работал утром, хоть и не говорил по-русски, был очень вежливый: Подключили к телефону WI FI, рассказали где находится ресторан и т.д.. Одним из плюсов этого отеля, это то что можно сходить на завтрак, даже если тебя пока не засели в номер. Точно также и было перед отъездом, ты можешь поесть и продолжить пользоваться услугами отеля, даже если ключи от номера сдал. У меня было «Все включено». Прочитала про отель, что он уже не новый. По факту он оказался аккуратный, все чисто и убрано. Номера очень уютные. Кормили вкусно и разнообразно, было много овощей. Всегда можно найти, что покушать и не остаться голодным. Приготовленная еда всегда была свежей. Также каждый день подавали еду на гриле: рыба, разные виды мяса. До моря идти пешком, не спеша минут 15, выделен отдельный пляж с зонтами и шезлонгами, также есть бар, в котором можно взять напитки. Еще от отеля предусмотрен автобус до пляжа, который отъезжает каждые 30 минут. Отель мне понравился, в своей ценовой категории оправдывает ожидания! Резюмируя отзыв, могу с уверенностью сказать, что отель оборудован и сделан так, как будто его делали для себя, что согласуется со словами из одной мудрой книгой «Во всем поступайте с людьми так, как хотите, чтобы они поступали с вами».', 'Отель очень понравился! Удобное расположение к морю, к Анталии. Комнату дали на 13 этаже с видом на Ricos5*.Еда прекрасная, всего полно, всё свежее и вкусное! Море тёплое, прозрачное, заход мелкая галька. Брали рафтинг, зиплайн, Тазы-каньон у alfaturizm. net. Очень хорошо! Спасибо, Турция, за чудесный отдых! До скорой встречи!', 'Начну с того , что в этом отеле мы отдыхали дважды в октябре 2022 г и в июне 2023 г. Долго сомневались в выборе именно этого отеля, прочитав противоречивые отзывы. Нам нужен был городской отель рядом с морем. Расположение отеля очень удачное, от аэропорта Анталии минут 30 езды. До центра Анталии минут 10 на такси ( около 50 лир). Так же рядом есть автобусная остановка и трамвайная. Море в пешей доступности по красивейшей набережной ( до пляжа отеля около 10-15 мин). Море чистое, плавный галечный вход. От отеля есть так же трансфер. Пляж оборудован бесплатными лежаками, зонтами. Безалкогольные напитки бесплатны. У отеля есть небольшая зеленая территория. Мебель в номерах не новая, постельное белье и полотенца всегда чистые. Имеются тапочки. В ванной комнате имеются банные принадлежности, фен. Сейф бесплатный. В минибаре вода , минералка, соки, иногда кола. Питание меня полностью устраивает. Все, что готовят на улице всегда все вкусно! Большое разнообразие блюд, овощей, закусок, несколько видов фруктов, супов. На завтрак иногда жарят пончики, блины ( на любителя). Вкусные десерты. Сейчас отдыхаю в Алании в Loxia beach comfort 5* и питание не сравниться с Ozkaymak Falez! Анимации практически в отеле нет. Один раз была живая музыка. Отель подойдет для тех , кто хочет совместить пляжный отдых с шоппингом, экскурсиями. Кто привык отдыхать в отелях уровнем выше, будут разочарованы, так как пятерка пятерке рознь. Нас отель полностью устраивает и планируем вновь посетить Ozkaymak Falez!', 'Самое лучшее сочетание цены и качества среди отелей в Анталье. В этот отель ездим с мамой с 2019 года. В апреле 2023 были в Озкаймаке уже седьмой раз. Прежде всего хочется выразить благодарность всему персоналу отеля: Мехмет, Сияра и Мария на ресепшене помогли решить все возникшие во время отдыха проблемы. Еще хочется отметить беллбоя Рамазана и бармена Фарруха - очень приветливые и солнечные)). В номерах убирают хорошо, в номере есть всё необходимое. Сам отель очень большой, с хорошим солидным дизайном. При отеле большая ухоженная зеленая зона с пальмами и разными декоративными деревьями. В ресторане кормят хорошо и вкусно - главное найти своё). Очень вкусная выпечка на завтрак. В баре у бассейна можно самому делать бургеры, есть всякие снэки. Будем рады вернуться сюда снова!)', 'Отдыхал в мае, 10 дней пролетели незаметно. Номер был большой, чистый и уютный. Очень очень много вкусной еды, невозможно оторваться. Спасибо команде развлечений за такие яркие моменты! Караоке-вечер это что-то) было очень весело) Особенная благодарность Nigth club и его персоналу😄', 'Очень чистые номера, к детям относятся с экстра вниманием, еда превосходная(и можно как до пандемии брать всё, что хочешь), за блюдами не стоят официанты и не чувствуешь себя некомфортно, спрашивая. Приветливые Акром и Шамшот помогают опредиться с ресторанами а-ля карт и сделать запись на ужин с ребёнком. Спасибо за отдых, он останется с нами в воспоминаниях. Будем копить, чтобы приехать с 3-мя детьми к вам. 5 из 5.', 'Отдыхали в отеле в период с 30.08.2023 по 08.09.2023 г. При раннем прибытии в отель (10 утра, при чек-ин 14:00) сразу зарегистрировали и дали браслет, пока готовился номер можно было позавтракать в основном ресторане. На ресепшене постоянно, кроме совсем раннего утра (7 утра), присутствует русскоговорящий сотрудник. Первоначально дали номер на первом этаже, при оплаченном виде на море - на первом этаже даже выходящем на сторону моря - этого самого моря не видно, мы не согласились с этим вариантом и попросили поменять номер на более высоком этаже, нашу просьбу выполнили и дали номер на 4 этаже, там уже вид был шикарен! Номер на троих не маленький и не большой, обычный, хорошо обустроен, приятная обстановка, одна двуспальная кровать, одна одноместная и диван (который тоже пригодился и на нем вполне было комфортно спать), в номере все необходимое: кондиционер, что примечательно - работал даже когда мы уходили и когда была открыта дверь; фен; телевизор - были русские каналы или нет мы не узнали, так как смотрели только Евроспорт; сейф; бар; чайник. Для размещения вещей трех взрослых людей места было достаточно, много плечиков, много полочек. По номеру были недостатки в виде сломанной ручки на двери балкона, сломана одна из сушилок для вещей на балконе. В целом номер и сам отель можно было бы обновить косметическим ремонтом. Что касается питания - 10 из 10, приходилось себя останавливать, чтобы не лопнуть от переедания, еда была доступна постоянно и везде. В ресторане всегда была рыба на гриле, невероятно вкусная ягнятина, были роллы, буррито, разные сыры и колбасы, пицца, клубника и ананасы в шоколаде, очень вкусный шоколад с орехами типа риттер спорт, мармелад, вообщем мы в восторге от того как кормят в этом отеле! Отель выходит прямо на море, есть свой пирс, лежаки и на пляже и на пирсе, пляж ухоженный, просторный, песчаный, вода в конце августа была как парное молоко, иначе не скажешь, видно рыбок, с пирса их кормили и разглядывали! Анимация и развлечения великолепные, каждый день новая программе, была и пенная и пивная вечеринки, дискотека в стиле Голливуд на открытом воздухе у бассейна, живая музыка в разных стилях и настроениях, много позитивных эмоций! Спортом мы тоже занимались, с 7 до 8 утра играли в большой теннис (ракетки и мячи под залог 10 евро), сразу скажу, если вы более менее продвинутый любитель и играете профессиональной ракеткой - лучше взять ее с собой, потому как там дают совсем дилетантские ракетки, если же вы никогда не играли в большой теннис - то ракетки вполне сойдут. Также большое спасибо за йогу, мы ходили почти каждый день, девушка, которая ее вела просто замечательная, к своему сожалению и стыду, мы даже не узнали как ее зовут( В бассейне мы не купались, поэтому не могу ничего сказать, только то что бассейны были вместительные и там всегда было место для всех желающих! Отель имеет шикарное озеленение, пальмы у бассейна, разные растения по всей округе, из моря смотрели, а из-за зелени была видна только макушка отеля, очень красиво, за территорией ежедневно ухаживали, подсаживали растения, большое спасибо за красивое оформление территории! На территории отеля работает «туристическая полиция») Сотрудники интересуются у гостей все ли им нравится, так мы сказали, что нас одолевает комар и прийдя в номер увидели, что стоят фумигаторы в розетках, очень приятно! Сладости мы закупали в большом супермаркете на противоположной стороне от отеля, он расположен слева от отеля, если стоять к нему спиной в 600 метрах, может четь подальше. В общем могу сказать, что отдыхом в этом отеле мы довольны на сто процентов и с чистой совестью можем порекомендовать его друзьям и знакомым)', 'хотим поблагодарить всю команду Delphin Palace за прекрасный отдых! отдельную благодарность хотим выделить менеджеру по работе с русскими туристами Ольге, если бы не она, то отдых не удался бы на все 100%, она как палочка выручалочка, ангел хранитель и вообще очень замечательный и светлый человек и ещё хотим выразить благодарность всем аниматорам детской и взрослой анимации, вы большие молодцы, с вами всегда было весело нам удалось отметить день рождения моего брата, все было шикарно, ребенок был в восторге, и за это тоже отдельное спасибо Ольге! все всегда готовы оказать помощь, сразу же придут, если нужна какая то помощь) также хотим сказать спасибо, работнику в основном ресторане, Хасану, всегда придёт, поможет донести еду до столика, спасибо большое) советуем этот отель, вы точно не пожалеете, приедем ещё раз в наш любимый и самый лучший Delphin Palace!!!', 'Отдыхали всей семьей в этом отеле не первый раз. Замечательный отель.хочется возвращаться сюда снова и снова. Отличное питание, расположение отеля, уют,чистота, развлечения. Все на высшем уровне. Хотелось отметить очень хорошее отношение всего персонала к гостям отеля. Особенно нам понравилась менеджер ОЛЬГА . Очень чуткий, отзывчивый , желающий понять и обязательно решить любой вопрос. Огромное спасибо ей и всему коллективу Вашего отеля за отличный отдых. До скорой встречи!!!', 'Отдыхаем с подругой в нашем уже любимом отеле второй год. Хочется отметить особое отношение персонала отеля к гостям. Встретили нас с радостью, с бокалом шампанского и вкусным лукумом, нам было очень приятно, что нас помнят. Отдельное спасибо Мехмету (Шеф ресепшн) за чувство юмора и понимание. Особую благодарность хочется выразить нашей дорогой Мехрибан( гест рилейшн), которая делала все, чтобы мы чувствовали себя максимально комфортно. Мы отдыхали в разных отелях в Турции, но более душевного отношения к гостям еще не встречали. При заселении в номере нас ждал приятный сюрприз, вино и фрукты. На что мы больше всего обратили внимание, это то, что персонал в этом отеле почти не поменялся, это о многом говорит. Чувствовали себя, как дома, хочется вернуться опять. Кухня великолепна, голодными не останетесь точно.Вечерние шоу на высоте, живая музыка просто супер! Попробуйте очень вкусный кофе по турецки, который делают в лобби баре. А вечером бармены сделают вам очень вкусный Special cocktail. Анимация супер! Каждый найдет занятие для себя по интересу. Ставим отелю 5 +++, хочется вернуться сюда снова❤️❤️❤️ С любовью к вам Татьяна и Инна', 'Большая территория, сервис на высшем уровне, еда вкусная , алкоголь оставляет желать лучшего, номер в первый раз дали без балкона, пришлось доплатить за номер лучше, ставим минус за это, потом дали хороший номер с доплатой, анимация супер,очень понравилась.', 'В отеле Fame Residence Lara отдыхаем уже второй раз и уверена, что вернёмся снова. Всё очень нравится, еда, территория, водные горки, анимация, навесы на пляже (вместо зонтов), номера - всё на 5 баллов. И от аэропорта близко, что тоже несомненный плюс. В этом году отель провёл реновацию, мы этого не знали и когда нас заселили в номер, где всё новенькое и свеженькое - это был как дополнительный приятный бонус. Единственный минус в этом году-это низкое качество интернета. Не знаю, что случилось, т.к. в прошлом году всё было нормально с интернетом. Об этой проблеме многие пишут, надеюсь это учтут, и в следующем году всё будет опять ок. Отдельное спасибо хочется сказать девушке из службы guest relation Мехрибан. Она свободно разговаривает по-русски. Уже второй год её помощь нам сложно переоценить. По всем вопросам она помогает так, как будто мы ей родные. Это очень приятно и ценно, когда ты в чужой стране. Когда уезжали, прямо не удержалась, обняла её, так я ей благодарна за её внимание, заботу и доброту! К остальным сотрудникам отеля тоже нет никаких нареканий, в номере убирались хорошо, в ресторане тоже было хорошее обслуживание. В общем всем от души рекомендую данный отель!', 'Балдёжный отель с отличным ландшафтом 👍 отдыхали семьей в сентябре с маленьким ребёнком. Сначала разместили в малюсенький номер без балкона с окном на дорогу, вид конечно красивый, но хотелось именно балкон и номер больше, попросили переселить, и нам дали отличный Фемели рум👍 территория чистая, я влюбилась в оформление около бассейна😍 пляж обычный, мы уходили на соседний пляж, тк там нам показалось чище и камней меньше. Еда разнообразная, пицца, раки, суши, мясо, кондитерка, и для детей питание 👍 детские стульчики. Днём удобно, что можно погулять по асфальту с коляской в теньке с ребёнком. Анимация на сцене, коктейли в баре, все отлично 👌 а ещё плюс, что относительно недалеко аэропорт и старый город👍', 'Отдыхали с сестрой с 20 мая по 9 июня 2023 года! Впервые приехали в этот отель в августе 2020 года после пандемии. Скажу так: "Если вам понравилось это место с первого взгляда - то вы обязательно вернётесь в Fame ещё не один раз!" Отдых начинается со входа в отель и бокала 🍾- релакс с первых минут пребывания! Ресепшен работает на "отлично" . Спасибо огромное Мехмету- решит любой вопрос в считанные минуты. Свободно говорит на нескольких языках! Отдельное спасибо службе помощи гостям и особенно милой хрупкой Мехрибан! Знает 5 языков как свой родной! Благодаря ей мы посетители 2 ужина в ресторанах аля карте- итальянский и морской ужин. Незабываемые впечатления от стейка!!! Особая благодарность шеф- повару Мустафе- профессионал своего дела!!! Изюминка отеля- кофейня! Кофе - божественный! Рекомендую! Всем официантам ресторанов особая благодарность- вы работаете с душой , как одна слаженная команда! Али, Эртан, Назим, Севги, Елена- ребята- вы лучшие! Шефы Аслан и Осман - профессионалы своего дела! Кухня разнообразная. Надеюсь, что даже самый привередливый отдыхающий не останется голодным. Особая любовь у меня к СПА-процедурам)) Улыбчивый Али уговорит любого порадовать себя массажем, пилингом или посещением хаммама)) Низкий поклон массажистке Нигаль- профессионал своего дела с "золотыми" руками!! В номерах - чисто, уютно, номер с видом на море- это просто сказка! Уборка номеров ежедневно , смена полотенец и постельного белья- без проблем. Наша горничная Гульсара на 7 этаже никого не оставит без улыбки и прекрасного настроения на весь день )) Пляжная полоса- просто супер! В сравнении с отелями- соседями- значительно шире, вход в море пологий, море- ласковое и волшебное! В общем- отель отличный, персонал адекватный и сделает всё, чтобы ваш отдых стал незабываемым и комфортным! Рекомендую!']


In [14]:
collect_words(positive_reviews2, all_pos_words) # добавляем к положительным словам слова из новых положительных отзывов

not_neaded_words = all_pos_words & all_neg_words  # заново ищем пересечение
only_pos_words = all_pos_words ^ not_neaded_words # находим новые уникальные слова
only_neg_words = all_neg_words ^ not_neaded_words

answers = []
for rev in test_reviews: # снова пытаемся определить тональность отзывов
    answers.append(determine_tonality(rev))

r = sklearn.metrics.confusion_matrix(right_answers, answers) # считаем accuracy
r = numpy.flip(r)
acc = (r[0][0] + r[-1][-1]) / numpy.sum(r)
print(acc)

0.9


In [15]:
print(len(only_neg_words))
print(len(only_pos_words))

1094
848


In [16]:
test_reviews[9]

'Администратор отеля не помог заселится ,по базе были ошибки .Заехали только в 16:30 🤦🏻\u200d♂️ Попросили нас подождать около 30 мин . Постели были грязные в зеркало сломанная . Администрацию оцениваю шкале 1 ,а персонала молодцы 5+.Фалкон 1511 бутик отель и Фалкон отель администраторы не смогли не как решить нашу проблему !'

Число уникальных слов для позитивных и негативных отзывов стало примерно одинаковым, и теперь правильно определяются 9 из 10 отзывов. Отзыв, который угадывается неверно, маленький, он посвящён конкретной проблеме с заселением, а не общей оценке отеля, что может быть причиной неверного определения его тональности.